In [110]:
import numpy as np
import pandas as pd
import plotly.express as px
import torch
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 * 10))
    series += 0.2 * np.sin((time  - offsets2) * (freq2 * 20 + 20))
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
    return series[..., np.newaxis].astype(np.float32)

In [26]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [27]:
series.shape

(10000, 51, 1)

In [28]:
px.line(x=list(range(len(series[0]))), y=series[0].ravel())

# Setting Up Torch

In [291]:
class Config:
    
    def __init__(self):
        self.ROOT_DATA_DIR = "FashionMNIST"
        self.EPOCH = 50
        self.BATCH_SIZE = 32
        self.LEARNING_RATE = 0.01
        self.IMAGE_SIZE = (28, 28)
        self.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"this notebook is using device : {self.DEVICE}")
        self.SEED = 2022
        
config = Config()

this notebook is using device : cpu


# Creating Data Loaders

In [141]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X)
        self.y = torch.tensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [142]:
train_data = CustomDataset(X_train, y_train)
valid_data = CustomDataset(X_valid, y_valid)
test_data = CustomDataset(X_test, y_test)

In [143]:
train_data_loader = DataLoader(
    dataset = train_data,
    batch_size = config.BATCH_SIZE,
    shuffle = True
)

test_data_loader = DataLoader(
    dataset = test_data,
    batch_size = config.BATCH_SIZE,
    shuffle = False
)

In [144]:
len(train_data_loader)

219

In [145]:
for x,y in train_data_loader:
    print(x.shape)
    print(y.shape)
    break

torch.Size([32, 50, 1])
torch.Size([32, 1])


In [323]:
class TimeSeries(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, in_features, out_features):
        super(TimeSeries, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.in_features = in_features
        self.out_features = out_features
        
        self.lstm_1 = nn.LSTM(input_size=self.input_size,
                              hidden_size=self.hidden_size,
                              num_layers=self.num_layers,
                              batch_first=True
                             )
        
        self.fc_01 = nn.Linear(in_features=self.in_features, out_features=self.out_features)
        
    def forward(self, x):
        h_0 = torch.autograd.Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        c_0 = torch.autograd.Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        output, (hn, cn) = self.lstm_1(x, (h_0, c_0))
        x = self.fc_01(output[:,:,0])
        return x
        

In [280]:
lstm_1 = nn.LSTM(input_size=1,hidden_size=1,num_layers=1,batch_first=True)

In [281]:
output, (hn, cn) = lstm_1(x)

In [321]:
output[:,:,0].shape

torch.Size([32, 50])

In [308]:
dense = nn.Linear(in_features=50, out_features=1)

In [322]:
dense(output[:,:,0])

tensor([[-0.1018],
        [-0.1035],
        [-0.0894],
        [-0.1026],
        [-0.1082],
        [-0.1177],
        [-0.1050],
        [-0.1095],
        [-0.1151],
        [-0.1029],
        [-0.1068],
        [-0.1044],
        [-0.1065],
        [-0.1025],
        [-0.0975],
        [-0.1125],
        [-0.0985],
        [-0.1108],
        [-0.0939],
        [-0.1205],
        [-0.1054],
        [-0.1275],
        [-0.1128],
        [-0.0943],
        [-0.1153],
        [-0.1271],
        [-0.1078],
        [-0.1115],
        [-0.0998],
        [-0.1178],
        [-0.1087],
        [-0.0976]], grad_fn=<AddmmBackward0>)

In [324]:
model = TimeSeries(
    input_size = 1,
    hidden_size = 1,
    num_layers = 2,
    in_features = 50,
    out_features = 1
)

In [325]:
print(model)

TimeSeries(
  (lstm_1): LSTM(1, 1, num_layers=2, batch_first=True)
  (fc_01): Linear(in_features=50, out_features=1, bias=True)
)


In [326]:
def count_params(model):
    model_params = {"Modules": list(), "Parameters": list()}
    total = {"trainable": 0, "non_trainable": 0}
    for name, parameters in model.named_parameters():
        # numel() returns the total no. of elements in the tensor
        param = parameters.numel()
        if not parameters.requires_grad:
            total["non_trainable"] += param
            continue
        model_params["Modules"].append(name)
        model_params["Parameters"].append(name)
        total["trainable"] += param

    df = pd.DataFrame(model_params)
    df = df.style.set_caption(f"Total parameters: {total}")
    return df

count_params(model) 

,Modules,Parameters
0,lstm_1.weight_ih_l0,lstm_1.weight_ih_l0
1,lstm_1.weight_hh_l0,lstm_1.weight_hh_l0
2,lstm_1.bias_ih_l0,lstm_1.bias_ih_l0
3,lstm_1.bias_hh_l0,lstm_1.bias_hh_l0
4,lstm_1.weight_ih_l1,lstm_1.weight_ih_l1
5,lstm_1.weight_hh_l1,lstm_1.weight_hh_l1
6,lstm_1.bias_ih_l1,lstm_1.bias_ih_l1
7,lstm_1.bias_hh_l1,lstm_1.bias_hh_l1
8,fc_01.weight,fc_01.weight
9,fc_01.bias,fc_01.bias


# Callbacks

In [332]:
class EarlyStopping():
    
    def __init__(self, patience=0):
        self.best_model = None
        self.best_loss = None
        self.patience = 0
        self.count = 0
        
    def __call__(self, model, loss):
        if model is None or self.best_loss > loss:
            self.best_model = model
            self.best_loss = loss
            self.count = 0
            return 0
        else:
            self.count += 1
            if self.count > self.patience:
                return -1
            

# Training model

In [333]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config.LEARNING_RATE)
early_stopping = EarlyStopping(patience = 8)

In [336]:
for epochs in range(10):
    with tqdm(train_data_loader) as tqdm_epoch:
        for x, y in tqdm_epoch:
            tqdm_epoch.set_description(f"Epoch {epochs + 1} / {config.EPOCH}")
            
            # Put images on device
            x = x.to(config.DEVICE)
            y = y.to(config.DEVICE)
            
            # Forward pass
            outputs = model(x)
            loss = criterion(outputs, y)
            net_loss.append(loss.item())
                
            
            # Backward pass
            optimizer.zero_grad() # sets all gradients to zero
            loss.backward() # calculate gradient of each weight
            optimizer.step() # updates the weights
            
            tqdm_epoch.set_postfix(loss=loss.item())
    

Epoch 1 / 50: 100%|█████████████████████████████████████████████████████| 219/219 [00:11<00:00, 19.73it/s, loss=0.0322]


0.02922157395937263


Epoch 2 / 50: 100%|█████████████████████████████████████████████████████| 219/219 [00:10<00:00, 20.27it/s, loss=0.0322]


0.02910504807807403


Epoch 3 / 50: 100%|█████████████████████████████████████████████████████| 219/219 [00:11<00:00, 19.82it/s, loss=0.0388]


...

0.029002305111724493


Epoch 9 / 50: 100%|█████████████████████████████████████████████████████| 219/219 [00:11<00:00, 19.65it/s, loss=0.0424]


0.02887162657616209


Epoch 10 / 50: 100%|████████████████████████████████████████████████████| 219/219 [00:10<00:00, 20.29it/s, loss=0.0279]

0.028762022450089998


# Testing our model

In [330]:
pred = np.array([])
target = np.array([])

with torch.no_grad():
    for batch, data in enumerate(test_data_loader):
        images = data[0].to(config.DEVICE)
        labels = data[1].to(config.DEVICE)
        
        
        y_pred = model(images)
        print(criterion(labels, y_pred).item())
        break
        pred = np.concatenate((pred, torch.argmax(y_pred, 1).cpu().numpy()))
        target = np.concatenate((target, labels.cpu().numpy()))

0.03282013535499573
